In [1]:
# Instalar as bibliotecas necessárias
!pip install pandas matplotlib statsmodels scipy pymannkendall ruptures scikit-learn

# Importar as bibliotecas necessárias
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from scipy.stats import kendalltau, linregress
import pymannkendall as mk
from ruptures import rpt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from sklearn.model_selection import TimeSeriesSplit
import os
from google.colab import drive
from multiprocessing import Pool

# Montar o Google Drive
drive.mount('/content/drive')

# Definir o diretório dos arquivos de dados corrigidos
directory_path = '/content/drive/MyDrive/DISSERT_ZANIN/EstudoVazao/VazaoCorrigidosGPT'

# Listar os arquivos no diretório especificado
file_list = os.listdir(directory_path)
file_list.sort()

# Função para processar cada arquivo
def process_file(file):
    if file.endswith('_corrigido.csv'):
        file_path = os.path.join(directory_path, file)

        # Carregar os dados de vazão corrigidos
        df = pd.read_csv(file_path, parse_dates=['Data'], dayfirst=True)
        df.set_index('Data', inplace=True)

        # Tratar dados faltantes
        df['Vazao'].fillna(method='ffill', inplace=True)

        # Visualizar a série temporal de vazão
        plt.figure(figsize=(14, 7))
        plt.plot(df.index, df['Vazao'], label='Vazão')
        plt.title(f'Vazão ao Longo do Tempo - {file}')
        plt.xlabel('Data')
        plt.ylabel('Vazão (m³/s)')
        plt.legend()
        plt.show()

        # Teste de Dickey-Fuller Aumentado para verificar estacionariedade
        adf_result = adfuller(df['Vazao'])
        print(f"Teste de Dickey-Fuller Aumentado para {file}:")
        print(f"ADF Statistic: {adf_result[0]}")
        print(f"p-value: {adf_result[1]}")

        # Teste de Mann-Kendall
        mk_result = mk.original_test(df['Vazao'])
        print(f"Teste de Mann-Kendall para {file}:")
        print(mk_result)

        # Regressão Linear para tendência
        slope, intercept, r_value, p_value, std_err = linregress(df.index.factorize()[0], df['Vazao'])
        print(f"Regressão Linear - Slope: {slope}, p-value: {p_value}")

        # Decomposição sazonal com diferentes períodos
        for period in [12, 365]:
            result = seasonal_decompose(df['Vazao'], model='additive', period=period)
            result.plot()
            plt.title(f'Decomposição Sazonal - Período {period} dias - {file}')
            plt.show()

        # Análise de Eventos Extremos com diferentes limiares
        thresholds = [0.90, 0.95, 0.99]
        for threshold in thresholds:
            extreme_events = df[df['Vazao'] > df['Vazao'].quantile(threshold)]
            plt.figure(figsize=(14, 7))
            plt.plot(df.index, df['Vazao'], label='Vazão Diária')
            plt.scatter(extreme_events.index, extreme_events['Vazao'], color='red', label=f'Eventos Extremos ({threshold*100}%)')
            plt.title(f'Eventos Extremos de Vazão - {file} - Limiar {threshold*100}%')
            plt.xlabel('Data')
            plt.ylabel('Vazão (m³/s)')
            plt.legend()
            plt.show()

        # Detecção de mudanças de ponto
        algo = rpt.Pelt(model="rbf").fit(df['Vazao'].values)
        change_points = algo.predict(pen=10)
        print(f"Pontos de mudança detectados: {change_points}")

        # Visualizar mudanças de ponto
        plt.figure(figsize=(14, 7))
        plt.plot(df.index, df['Vazao'], label='Vazão Diária')
        for cp in change_points:
            plt.axvline(df.index[cp], color='r', linestyle='--', label='Mudança de Ponto')
        plt.title(f'Mudanças de Regime na Vazão - {file}')
        plt.xlabel('Data')
        plt.ylabel('Vazão (m³/s)')
        plt.legend()
        plt.show()

        # Análise de Autocorrelação
        plot_acf(df['Vazao'], lags=50)
        plt.title(f'Função de Autocorrelação (ACF) - {file}')
        plt.show()

        plot_pacf(df['Vazao'], lags=50)
        plt.title(f'Função de Autocorrelação Parcial (PACF) - {file}')
        plt.show()

        # Validação Cruzada
        tscv = TimeSeriesSplit(n_splits=5)
        for train_index, test_index in tscv.split(df):
            train, test = df.iloc[train_index], df.iloc[test_index]
            # Realizar análises no conjunto de treino e validar no conjunto de teste
            print(f"Treino: {train.index.min()} a {train.index.max()}")
            print(f"Teste: {test.index.min()} a {test.index.max()}")

        # Exportar gráficos
        plt.savefig(f'/content/drive/MyDrive/DISSERT_ZANIN/EstudoVazao/Resultados/{file}_vazao.png')

        # Exportar resultados para CSV
        df.to_csv(f'/content/drive/MyDrive/DISSERT_ZANIN/EstudoVazao/Resultados/{file}_resultados.csv')

# Usar multiprocessamento para processar os arquivos em paralelo
with Pool() as pool:
    pool.map(process_file, file_list)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 22.6 MB/s eta 0:00:00


ImportError: cannot import name 'rpt' from 'ruptures' (/usr/local/lib/python3.11/dist-packages/ruptures/__init__.py)